# ont_rdb_explorer

## Import packages
Also set the path to ont_rdb and the name of your snakemake profile for processing purposes.

In [12]:
import informant_class

# Adjust this to the directory storing the ont_rdb package.
ont_rdb_path = "/home/cfrankston/Projects/ont_rdb/ont_rdb"

# Set the snakemake profile you wish to use:
snakemake_profile = 'mamba'

from IPython.display import display, clear_output
import ipywidgets as widgets
import os
import importlib
import sys

## Select and import ontology module from drop-down menu.


In [13]:

selected_module_name = None
imported_module = None

def list_files_in_folder(folder_path):
    """Lists files in the given folder path."""
    files = [f for f in os.listdir(folder_path) if (os.path.isfile(os.path.join(folder_path, f)) and f.endswith("_ontology.py"))]
    return files

ontologies_folder = ont_rdb_path + '/ontologies'
file_list = list_files_in_folder(ontologies_folder)
dropdown_menu = widgets.Dropdown(options=file_list, description='Files:', disabled=False)
display(dropdown_menu)

button = widgets.Button(description="Import Selected Ontology Module", layout=widgets.Layout(width='auto'))
display(button)

output = widgets.Output()

@button.on_click
def button_on_click(b):
    global selected_module_name
    global imported_module
    with output:
        clear_output()
        selected_file = dropdown_menu.value
        module_name = selected_file[:-3]  # Remove the '.py' extension
        import_path = f'ontologies.{module_name}'
        
        try:
            # Dynamically import the selected module
            imported_module = importlib.import_module(import_path)
            # Optionally, add the imported module to sys.modules
            sys.modules[module_name] = imported_module
            print(f"Successfully imported {module_name}.")
            selected_module_name = module_name
        except Exception as e:
            print(f"Failed to import {module_name}: {e}.")

display(output)



Dropdown(description='Files:', options=('hic_January_24_2024_ontology.py',), value='hic_January_24_2024_ontolo…

Button(description='Import Selected Ontology Module', layout=Layout(width='auto'), style=ButtonStyle())

Output()

### Import informant classes from ontology module and construct or access its associated digraph dataframe.
If necessary, modify the ``command`` to use appropriate snakemake configurations.

For example:
* ``use-conda: true``
* ``conda-frontend: mamba``
* ``cores: 1``

In [14]:
import pandas as pd
import subprocess
import snakemake
command = f"snakemake --profile {snakemake_profile} ontology_dataframes/{selected_module_name}_dataframe.pkl"
result = subprocess.run(command, capture_output=True, text=True, shell=True)
# Check if the command was successful
if result.returncode == 0:
    # Print the standard output of the command
    print(f"{selected_module_name} dataframe is constructed.", result.stdout)
else:
    # Print the standard error if the command failed
    print("Command failed with error:", result.stderr)

ontology_dataframe = pd.read_pickle(ont_rdb_path + '/ontology_dataframes/' + selected_module_name + '_dataframe.pkl')

selected_module = importlib.import_module(selected_module_name)
for name in dir(selected_module):
    if not name.startswith('_'):  # Skip internal names
        globals()[name] = getattr(selected_module, name)
# ontology_dataframe

hic_January_24_2024_ontology dataframe is constructed. 


## Explore your ontology and create your database.
Construct and save informants and informant dataframes to organize  objects in the context of your ontology. 

In [11]:
my_informant_dataframe = informant_class.Informant_Dataframe()

bedpe_inf = BedPe_File()
bedpe_inf.__dict__
bedpe_inf.__dict__.update({'name':"ENCFF661SAZ.bedpe",
'description': 'Basic loops file from ENCODE.',
'species': 'homo_sapiens',
'location': "/home/cfrankston/Projects/Auxiliaries/bedpe_tools/bedpe_data/ENCFF661SAZ.bedpe",
'genome_assembly_name': "GRCh38",
'gz':False})

bedpe_inf.__dict__

my_informant_dataframe.append([bedpe_inf])
my_informant_dataframe.df

my_informant_dataframe.filter("(@genome_assembly_name == 'GRCh38') & (@gz == False)")

,name,informant,entry_time,verification_status
0,ENCFF661SAZ.bedpe,<ontologies.hic_January_24_2024_ontology.BedPe...,03_01_2024,pending
